In [ ]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import accuracy_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
    

In [ ]:

# Define file paths
script_dir = 'D:/Projects/agriculture/agri_backend/Train/DataSets/'

# Load datasets
crop_rec_df = pd.read_csv(script_dir + 'Crop_recommendation.csv')
crop_data_df = pd.read_csv(script_dir + 'crop_data.csv')
fertilizer_df = pd.read_csv(script_dir + 'Fertilizer.csv')

# Strip spaces and make all column names lowercase
crop_rec_df.columns = crop_rec_df.columns.str.strip().str.lower()
crop_data_df.columns = crop_data_df.columns.str.strip().str.lower()
fertilizer_df.columns = fertilizer_df.columns.str.strip().str.lower()

# Normalize crop column to lowercase
crop_rec_df["crop"] = crop_rec_df["label"].str.strip().str.lower()
crop_data_df["crop"] = crop_data_df["crop"].str.strip().str.lower()

# Merge datasets
combined_df = pd.merge(crop_rec_df, crop_data_df, on="crop", how="inner")
combined_df = pd.merge(combined_df, fertilizer_df, on="crop", how="left")

# Clean up missing data
combined_df = combined_df.dropna()

# Check for duplicates
combined_df = combined_df.drop_duplicates()
combined_df.head()
    

In [ ]:

# Prepare features and target variables
X = combined_df.drop(['crop', 'label'], axis=1)
y = combined_df['label']

# Encode target variable (label) as numerical values
le = LabelEncoder()
y_encoded = le.fit_transform(y)
y_categorical = to_categorical(y_encoded)

# Standardize features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_categorical, test_size=0.2, random_state=42)
    

In [ ]:

# Build the Deep Neural Network model
model = Sequential([
    Dense(128, activation='relu', input_shape=(X.shape[1],)),
    Dropout(0.3),
    Dense(64, activation='relu'),
    Dropout(0.3),
    Dense(y_categorical.shape[1], activation='softmax')
])

# Compile the model
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
    

In [ ]:

# Train the model
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.1, verbose=1)
    

In [ ]:

# Evaluate the model on the test data
y_pred_probs = model.predict(X_test)
y_pred = np.argmax(y_pred_probs, axis=1)
y_true = np.argmax(y_test, axis=1)

# Calculate accuracy
accuracy = accuracy_score(y_true, y_pred)
print(f'Model Accuracy: {accuracy * 100:.2f}%')
    

In [ ]:

# Plot the training and validation loss
plt.figure(figsize=(10, 6))
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

# Plot the training and validation accuracy
plt.figure(figsize=(10, 6))
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()
    